In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/Ziqi/Desktop/mgwr-optim/')
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

C:\ProgramData\Anaconda3\lib\site-packages\libpysal\io\iohandlers\__init__.py:25: UserWarning: SQLAlchemy and Geomet not installed, database I/O disabled
  warnings.warn('SQLAlchemy and Geomet not installed, database I/O disabled')


In [2]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [3]:
df = pd.read_csv('/Users/Ziqi/Desktop/MGWR computational improvements/Irish Famine Variables2010_05_04 without Ulster.csv')
df.head()

,OBJECTID,X,Y,Percentchange,PPB,Percent_urban_41,Ratio1841,PCorn_Cult,UNINHABPCT,MEAN_ELEV,lnPopCrop,lnPotCult,lnWorkhouse,lnAHS,lnDistCoast,lnUrbanAcc
0,874,128046.76,210113.45,-34.744409,6.731183,15.694888,104.742,51.640112,2.872063,21.560,0.876747,3.216060,-2.708330,3.465529,2.363688,7.261111
1,875,131157.13,200881.19,-36.842105,6.100917,0.000000,106.522,38.364780,2.678571,195.441,1.703997,3.002002,-2.900186,4.952171,9.077177,7.332147
2,876,125931.88,197947.14,-37.237762,6.426966,0.000000,111.070,35.714286,0.000000,255.351,1.764171,3.182508,-2.692713,4.930321,9.198086,7.293188
3,877,113204.66,204113.08,-51.907514,6.006944,0.000000,104.009,46.706587,0.689655,124.303,1.745450,2.854472,-3.098829,4.614020,6.993490,7.105138
4,879,116550.29,208855.28,-37.841727,5.864979,0.000000,100.577,43.893130,1.659751,154.058,1.703669,3.226480,-2.985109,4.065788,7.596820,7.006520


In [4]:
y = df.Percentchange.values.reshape(-1,1)
X = df[['lnPopCrop','PCorn_Cult','lnWorkhouse','lnUrbanAcc','lnPotCult','lnDistCoast','MEAN_ELEV']].values
k = df.shape[1]
u = df.X
v = df.Y
n = df.shape[0]
coords = np.array(list(zip(u,v)))
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean(axis=0)) / y.std(axis=0)

In [5]:
import multiprocessing as mp
pool=mp.Pool()

### GWR

In [6]:
%%time
bw = Sel_BW(coords,y,X).search(pool=pool)
rslt = GWR(coords,y,X,bw=bw).fit(pool=pool)

Wall time: 6.79 s


### MGWR Backfitting

In [7]:
%%time
se = Sel_BW(coords,y,X,multi=True)
bws = se.search(pool=pool)

Wall time: 6min 3s


In [8]:
bws

array([  43.,  314.,  310.,  439., 2275., 1385., 1321., 2178.])

### MGWR Inference

In [9]:
%%time
rslt = MGWR(coords, y, X, selector=se).fit(pool=pool)

Wall time: 6min 34s


In [10]:
pool.close()
pool.join()